<a href="https://colab.research.google.com/github/LucaChuang/Taiwan-Stock-Information/blob/main/Taiwan_stock_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import datetime
import requests
import pandas as pd
import time
warnings.filterwarnings("ignore")

In [36]:
headers={
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'
    }
res = requests.get('https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date=20151001&stockNo=3037', headers = headers)
res.encoding = 'utf-8'
#res.text

In [38]:
from datetime import date,timedelta
from dateutil import rrule
def till_now(start_month):
    b_month = date(*[int(x) for x in start_month.split('-')])
    now = datetime.datetime.now().strftime("%Y-%m-%d")         
    e_month = date(*[int(x) for x in now.split('-')])
    result = []
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=b_month, until=e_month):
              result += [int(dt.strftime('%Y%m%d'))]
    return result


In [50]:
def stockinfo(begin_date, stock_no):
  dates = test = till_now(begin_date) # 2021-01-01
  stock_no = stock_no
  df_stock_info =pd.DataFrame()
  url_template = "https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=html&date={}&stockNo={}"
  for date in dates :
    time.sleep(3)
    url = url_template.format(date, stock_no)
    data = pd.read_html(requests.get(url).text)[0]
    data.columns = data.columns.droplevel(0)
    df_stock_info = pd.concat([df_stock_info,data])
    df_stock_info = df_stock_info.reset_index(drop = True)
    print("目前進度: "+str(date))  
  df_temp = pd.DataFrame(df_stock_info["日期"].str.split("/").tolist(),
                               columns = ['year','mon', "date"])
  df_temp['year'] = df_temp['year'].astype("int32")+1911
  df_stock_info["日期"] = df_temp['year'].astype("string") + "-" + df_temp['mon'] + "-" + df_temp['date']
  file_name = str(stock_no)+" stock info from " + str(begin_date)
  df_stock_info.to_csv('/content/drive/My Drive/'+file_name+".csv", index=False)        #匯出csv檔
  df_stock_info.to_excel('/content/drive/My Drive/'+file_name+".xlsx", index=False)     #匯出excel檔
  return df_stock_info


In [51]:
if __name__ == '__main__':
    # -------- configurable parameter -------- #
    
    begin_date = "2020-11-01"           # 設定起始日期 ("2020-11-01")
    stock_no = 2330                   # 設定股票代碼 (2330)
    # ---------------------------------------- #
    stockinfo(begin_date, stock_no)

    print('\nfinish!')




目前進度: 20201101
目前進度: 20201201
目前進度: 20210101
目前進度: 20210201
目前進度: 20210301

finish!
